In [9]:
import pandas as pd
import os

In [22]:
# Load Data
risk_factors_df = pd.read_excel('data/risk-factors.xlsx')
risk_categories_df = pd.read_excel('data/risk-factors-categories.xlsx')

print("risk factors shape:", risk_factors_df.shape)
print("risk categories shape:", risk_categories_df.shape)



risk factors shape: (167, 2)
risk categories shape: (167, 2)


In [29]:
# As length of keyword lists is identical we can use set to check if the content is also identical 
# Using python set (as it can only contain uniques) varialbe for quick comparision. 
# If lenght of set would differ from length of original dataframe we would need to find a different way. 

# Extracting risk factors, convert to lowercase, and remove leading/trailing whitespace just in case
factors_set = set(risk_factors_df['risk_factor_english'].dropna().str.lower().str.strip())
categories_set = set(risk_categories_df['risk_factor'].dropna().str.lower().str.strip())

print(f"Number of unique keywords in risk-factors.xlsx: {len(factors_set)}")
print(f"Number of unique keywords in risk-factors-categories.xlsx: {len(categories_set)}")

# Find differences
common_keywords = factors_set.intersection(categories_set)
only_in_factors = factors_set - categories_set
only_in_categories = categories_set - factors_set

print(f"  - Keywords in both files: {len(common_keywords)}")
print(f"  - Keywords only in risk-factors.xlsx: {len(only_in_factors)}")
print(f"  - Keywords only in risk-factors-categories.xlsx: {len(only_in_categories)}")
print("\nKeywords only in 'risk-factors.xlsx'", only_in_factors)
print("\nKeywords only in 'risk-factors-categories.xlsx' (examples):", only_in_categories)


Number of unique keywords in risk-factors.xlsx: 167
Number of unique keywords in risk-factors-categories.xlsx: 167
  - Keywords in both files: 166
  - Keywords only in risk-factors.xlsx: 1
  - Keywords only in risk-factors-categories.xlsx: 1

Keywords only in 'risk-factors.xlsx' {'"d\\\\etat"'}

Keywords only in 'risk-factors-categories.xlsx' (examples): {"d'etat"}


In [31]:
# okay so we clean up this little inconsisteny and make a note for the cleaning function in src later
def clean_keywords(keyword_series):
    """Basic cleaning steps from above as well as handling the special cases we found"""
    cleaned = keyword_series.dropna().str.lower().str.strip()
    # Replace '\\' with "'" and remove '"' characters. NOTE: Do not reuse as this will replace all backslashes with this special case here.
    # Using regex=False for literal replacement.
    cleaned = cleaned.str.replace('\\\\', "'", regex=False)
    cleaned = cleaned.str.replace('"', '', regex=False)
    return cleaned


In [35]:
# Apply the cleaning function to the keyword columns
factors_set_cleaned = set(clean_keywords(risk_factors_df['risk_factor_english']))
categories_set_cleaned = set(clean_keywords(risk_categories_df['risk_factor']))


# Re-compare
common_keywords = factors_set_cleaned.intersection(categories_set_cleaned)
only_in_factors = factors_set_cleaned - categories_set_cleaned
only_in_categories = categories_set_cleaned - factors_set_cleaned

print(f"  - Keywords in both files: {len(common_keywords)}")
print(f"  - Keywords only in risk-factors.xlsx: {len(only_in_factors)}")
print(f"  - Keywords only in risk-factors-categories.xlsx: {len(only_in_categories)}")

  - Keywords in both files: 167
  - Keywords only in risk-factors.xlsx: 0
  - Keywords only in risk-factors-categories.xlsx: 0


# Conclusion
My conclusion is that the risk-factors.xlsx is redundant. The only difference was one string deviation. I will use only the risk-factors-categories from now and will continue to do the arabic translation there to have a full dataset to work with.


# Next Step: 
I am now importing the translated dataframe we got from our translation.ipynb and continue explore and extract the clusters accordingly such that we can focus on the anaylsis

In [99]:
# Import translated data and create bilingual cluster structure

translated_df = pd.read_excel('new_data/risk-factors-translated.xlsx')

# Map English risk factors to clusters using the existing mapping
english_to_cluster = dict(zip(risk_categories_df['risk_factor'], risk_categories_df['cluster']))

# Add cluster information to translated data
translated_df['cluster'] = translated_df['risk_factor_english'].map(english_to_cluster)


In [105]:
# Group by cluster and collect both English and Arabic keywords + cluster names
bilingual_clusters = translated_df.groupby('cluster').agg({
    'risk_factor_english': list,
    'risk_factor_arabic': list,
    'cluster_english': list,
    'cluster_arabic': list
}).reset_index()

# Create clean cluster summary
cluster_summary_bilingual = []
for _, row in bilingual_clusters.iterrows():
    cluster_summary_bilingual.append({
        'cluster_english': row['cluster'],
        'cluster_arabic': row['cluster_arabic'][0],  # Arabic cluster name (same for all in cluster)
        'english_keywords': row['risk_factor_english'],
        'arabic_keywords': row['risk_factor_arabic'],
        'keyword_count': len(row['risk_factor_english'])
    })

bilingual_cluster_df = pd.DataFrame(cluster_summary_bilingual).sort_values('keyword_count', ascending=False).reset_index(drop=True)

bilingual_cluster_df.head(5)

,cluster_english,cluster_arabic,english_keywords,arabic_keywords,keyword_count
0,conflicts and violence,الصراعات والعنف,"[clan battle, foreign troops, conflict, violen...","[معركة العشيرة, القوات الأجنبية, الصراع, القمع...",35
1,political instability,عدم الاستقرار السياسي,"[oppressive regimes, anti-western policies, la...","[الأنظمة القمعية, سياسات معادية للغرب, غياب ال...",19
2,economic issues,القضايا الاقتصادية,"[slashed export, price rise, reduced national ...","[تصدير مخفض, ارتفاع الأسعار, انخفاض الناتج الق...",16
3,humanitarian aid,المعونة الإنسانية,"[withheld relief, international alarm, humanit...","[الإغاثة المحتجزة, الإنذار الدولي, الحالة الإن...",16
4,agricultural production issues,قضايا الإنتاج الزراعي,"[failed crops, disruption to farming, harvests...","[المحاصيل الفاشلة, تعطيل الزراعة, الحصاد مدمّر...",15


In [107]:
# Save to Excel file
output_path = 'new_data/bilingual_clusters.xlsx'
bilingual_cluster_df.to_excel(output_path, index=False)
print(f"Saved translated data to {output_path}")

# Verify the file was saved correctly
verification_df = pd.read_excel(output_path)
verification_df.head(5)


Saved translated data to new_data/bilingual_clusters.xlsx


,cluster_english,cluster_arabic,english_keywords,arabic_keywords,keyword_count
0,conflicts and violence,الصراعات والعنف,"['clan battle', 'foreign troops', 'conflict', ...","['معركة العشيرة', 'القوات الأجنبية', 'الصراع',...",35
1,political instability,عدم الاستقرار السياسي,"['oppressive regimes', 'anti-western policies'...","['الأنظمة القمعية', 'سياسات معادية للغرب', 'غي...",19
2,economic issues,القضايا الاقتصادية,"['slashed export', 'price rise', 'reduced nati...","['تصدير مخفض', 'ارتفاع الأسعار', 'انخفاض النات...",16
3,humanitarian aid,المعونة الإنسانية,"['withheld relief', 'international alarm', 'hu...","['الإغاثة المحتجزة', 'الإنذار الدولي', 'الحالة...",16
4,agricultural production issues,قضايا الإنتاج الزراعي,"['failed crops', 'disruption to farming', 'har...","['المحاصيل الفاشلة', 'تعطيل الزراعة', 'الحصاد ...",15


In [108]:
# Save as csv
output_path = 'new_data/bilingual_clusters.csv'
bilingual_cluster_df.to_csv(output_path, index=False)
print(f"Saved translated data to {output_path}")


Saved translated data to new_data/bilingual_clusters.csv
